# Import Libraries

In [11]:
import os
from raw_utils import save_to_csv #reference:https://github.com/liakoyras/thesis-phishing-email-detection
import preprocessing as util #reference:https://github.com/liakoyras/thesis-phishing-email-detection
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import re
import spacy
import pandas as pd
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer,SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


# Importing the Cleaned Dataset

In [12]:
# Load the datasets,and set the datatyoes of each column
balanced = pd.read_csv('balanced.csv',index_col=0,dtype={'body': 'object', 'class': 'bool', 'id': 'int16'})
balanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3836 entries, 0 to 3835
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      3836 non-null   int16 
 1   body    3835 non-null   object
 2   class   3836 non-null   bool  
dtypes: bool(1), int16(1), object(1)
memory usage: 71.2+ KB


# Basic Manipulation

In [5]:
#Show the head structure of the first 10 rows
balanced.head(10)

#print the content of the first index using the iloc function
body_bal=(balanced['body'].iloc[1])
class_bal=(balanced['class'].iloc[1])

print(class_bal,body_bal)

False 

_____________________DAVID COURSEY_____________________

HO-HO-HOW TO BUY A COMPUTER FOR CHRISTMAS

    It's that time of year again! If you're considering
         buying yourself a spiffy new PC (maybe spiced
         up with a broadband Internet connection?)
         or are looking to bestow technological riches
         on someone else this season, don't miss this
         great advice from Santa David.

http://cgi.zdnet.com/slink?/adeskb/adt1126/2826742:8593142


#


_____________________NEWS ANALYSIS_____________________

Sylvia Carr

    SONY ON DVD FENCE... NEW LOOK FOR LINUX... MS SERVERS
ATTACKED...

    The battle over which DVD format will
    become the industry standard wages
    on. Sony's weighed in on which one it
    prefers, but its stance is hardly decisive.
    Also, a new graphical interface for
    Linux and other Unix-like OS's has arrived,
    and a worm is wreaking havoc on Microsoft
    SQL servers.

http://cgi.zdnet.com/slink?/adeskb/adt1126/2827001:

# Preprocessing Step

In [6]:
# Define preprocessing functions
def clean_text(text):
    """
    Cleans text data by removing special characters, digits, and converting to lowercase.
    """
    if text is None or not isinstance(text, str):
        return ""
    # Remove special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Remove HTML tags
    #text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    return text

In [7]:
#Applying the various pre-processing functions;Stop word removal,punctuation, lemmentization and Tokenize the text


# Apply the clean_text function to the email and comment bodies(social media dataset)
balanced['body'] = balanced['body'].apply(lambda x: clean_text(x))

# Tokenize the email and comment bodies into individual words
balanced['tokens'] = balanced['body'].apply(lambda x: word_tokenize(x))

# Remove stop words and punctuation from the tokenized words
stop_words = set(stopwords.words('english'))
punctuation = set(['.', ';', ':', '!', '?', '~', '-', '(', ')', '[', ']', '{', '}', '<', '>', '/', '\\', '"', '%', '#', '@', "'", '+', '`', '£', '=', '$'])
balanced['tokens'] = balanced['tokens'].apply(lambda x: [word for word in x if word not in stop_words and word not in punctuation])


#Perform lemmatization and stemming to reduce the words to their base forms
lemmatizer = WordNetLemmatizer()
#stemmer = PorterStemmer()
balanced['tokens'] = balanced['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [9]:
#PRINT THE RESULTS OF THE DATA PREPROCESSING TASK

print("\033[1m" + "This is the Token column:" + "\033[0;0m",'\n',balanced['tokens'].iloc[1])
print('\n')
print("\033[1m" + "This is the original body column:" + "\033[0;0m",'\n',balanced['body'].iloc[1])

This is the Token column: 
 ['_____________________david', 'coursey_____________________', 'hohohow', 'buy', 'computer', 'christmas', 'time', 'year', 'youre', 'considering', 'buying', 'spiffy', 'new', 'pc', 'maybe', 'spiced', 'broadband', 'internet', 'connection', 'looking', 'bestow', 'technological', 'rich', 'someone', 'else', 'season', 'dont', 'miss', 'great', 'advice', 'santa', 'david', 'httpcgizdnetcomslinkadeskbadt', '_____________________news', 'analysis_____________________', 'sylvia', 'carr', 'sony', 'dvd', 'fence', 'new', 'look', 'linux', 'm', 'server', 'attacked', 'battle', 'dvd', 'format', 'become', 'industry', 'standard', 'wage', 'sonys', 'weighed', 'one', 'prefers', 'stance', 'hardly', 'decisive', 'also', 'new', 'graphical', 'interface', 'linux', 'unixlike', 'os', 'arrived', 'worm', 'wreaking', 'havoc', 'microsoft', 'sql', 'server', 'httpcgizdnetcomslinkadeskbadt', '_____________________expert', 'advice_____________________', 'lee', 'schlesinger', 'patchwork', 'approach', 

In [9]:
csv_path = os.path.join('/Users/mimidubcys/Desktop/Projects/A.I Powered Phishing Detection System/Project Files/System Architecture/Models/Model Building-Balanced/Balanced Dataset task')
save_to_csv(balanced, csv_path, 'balanced_processed_nostem.csv')

Saving to /Users/mimidubcys/Desktop/Projects/A.I Powered Phishing Detection System/Project Files/System Architecture/Models/Model Building-Balanced/Balanced Dataset task/balanced_processed_nostem.csv
